<a href="https://colab.research.google.com/github/ciaoCiao100/proveChallenge2/blob/master/customerTensorflowProva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

vengono importate le librerie , scaricato il dataset e impostata la target label

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# operazione che fa in modo che i numeri siano più leggibili
np.set_printoptions(precision=3, suppress=True)

data_path ="/content/drive/My Drive/WA_Fn-UseC_-Telco-Customer-Churn.train.csv"
df = pd.read_csv(data_path)
target_label ="Churn"
df.head()


,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,No,4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,Yes,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8
1,No,6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45
2,Yes,3797-VTIDR,Male,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45
3,Yes,2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95
4,No,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,


conversione degli elementi presenti in tabella in interi 

In [16]:
#tutte le colonne che contengono stringhe devono essere trasformate in colonne che contengono interi 
for elemento in df:
 
  df[elemento] = pd.Categorical(df[elemento])
  df[elemento] = getattr(df, elemento).cat.codes
  if elemento=="MonthlyCharges" or elemento=="TotalCharges" or elemento=="tenure":
   df[elemento] = df[elemento].astype('float')
  else:
   df[elemento] = (pd.Categorical(df[elemento])).codes.astype('float')
# mostro il dataset dopo l'operazione
df.head()


,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0.0,2370.0,0.0,0.0,0.0,1.0,21.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0,3.0,586.0,492.0
1,0.0,3457.0,0.0,0.0,0.0,0.0,54.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,2.0,2.0,2.0,1.0,0.0,1170.0,3511.0
2,1.0,2149.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,59.0,1566.0
3,1.0,1426.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,674.0,1627.0
4,0.0,1545.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0,546.0,0.0


preparazione del modello della rete

In [18]:
#costruisco il modello
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

model = get_compiled_model()
#preparo il dataset
train, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)

addestramento

In [19]:
# Creazione  del training
train_target = train.pop(target_label)
train_dataset = (tf.data.Dataset.from_tensor_slices((train.values, train_target.values))).shuffle(len(train)).batch(1)
#addestramento e impostazione epoche
model.fit(train_dataset, epochs=30)

Epoch 1/30

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

4563/4563 [==============================] - 5s 1ms/step - loss: 16.9615 - accuracy: 0.6603
Epoch 2/30
4563/4563 [==============================] - 5s 1ms/step - loss: 2.5090 - accuracy: 0.7015
Epoch 3/30
4563/4563 [==============================] - 5s 1ms/step - loss: 1.3632 - accuracy: 0.7278
Epoch 4/30
4563/4563 [==============================] - 5s 1ms/step - loss: 1.1996 - accuracy: 0.7197
Epoch 5/30
4563/4563 [==============================] - 5s 1ms/step - loss: 0.9226 - accuracy: 0.74

test

In [20]:
# Creazione del  test 
test_target = test.pop(target_label)
test_dataset = (tf.data.Dataset.from_tensor_slices((test.values, test_target.values))).shuffle(len(test)).batch(1)

# Fase di test
test_loss, test_accuracy = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

564/564 [==============================] - 1s 909us/step - loss: 0.4407 - accuracy: 0.7465


Test Loss 0.4407014548778534, Test Accuracy 0.7464538812637329


validazione

In [21]:
#valutazione
val_target = val.pop(target_label)

val_dataset = (tf.data.Dataset.from_tensor_slices(val.values)).batch(1)

# Fase di valutazione
predictions = model.predict(val_dataset)


# Show some results 
for prediction, yes in zip(predictions[:10], list(val_target)[:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted yes: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("yes" if bool(yes) else "no"))


Predicted yes: 1.62%  | Actual outcome:  no
Predicted yes: 56.65%  | Actual outcome:  yes
Predicted yes: 4.45%  | Actual outcome:  no
Predicted yes: 13.82%  | Actual outcome:  no
Predicted yes: 8.74%  | Actual outcome:  no
Predicted yes: 11.03%  | Actual outcome:  no
Predicted yes: 3.23%  | Actual outcome:  no
Predicted yes: 14.55%  | Actual outcome:  no
Predicted yes: 22.02%  | Actual outcome:  no
Predicted yes: 38.64%  | Actual outcome:  no
